## Table Of Contents

0. [References](#Reference)
1. [Getting Started](#GettingStarted)
2. [Architecture](#Architecture)
3. [Masked Convolution](#MaskedConvolution)
4. [First Masked Convolution](#FirstMaskedConvolution)
    1. [Keras.Input](#KerasInput)
5. [Residual Blocks](#ResidualBlocks)
    1. [Prelu details](#PRelu)
6. [Stacking Residual Blocks](#StackingResidualBlocks)
7. [Wrapping Up For Outputs](#WrappingUpForOutput)
8. [Output](#Output)
9. [Compiling the Model](#CompilingTheModel)
10. [Training the Model](#TrainingTheModel)
    1. [Loading MNIST data with Keras](#LoadingMNIST)
    2. [Fit the model to MNIST](#FitMNIST)
    3. [Generating MNIST](#GenerateMNIST)

<a id="References"></a>
## References

This code has been implemented following the steps and instructions provided in this [link](https://israelg99.github.io/2017-02-27-Grayscale-PixelCNN-with-Keras/) which in turn is based on the [paper](https://arxiv.org/pdf/1601.06759.pdf).

<a id="GettingStarted"></a>
## Getting Started

Keras has two ways of defining models, the Sequential, which is the easiest but limiting way, and the Functional, which is more complex but flexible way.

We will use the Functional API because we need that additional flexibility, for example - the Sequential model limits the amount of outputs of the model to 1, but to model RGB channels, we will need 3 output units, one for each channel. As the model gets more complex (e.g Gated PixelCNN) it will become clearer why Functional API is a no-brainer for projects like this.

Our input shape(excluding batch) should be: (height, width, channels).
More specifically, MNIST (grayscale) input shape looks like this (28, 28, 1) and CIFAR (32, 32, 3).

Let’s start simple, we’ll do a PixelCNN for grayscale MNIST first.

<a id="Architecture"></a>
## Architecture

Since the paper focuses on PixelRNN, it fails to provide a clear explanation on how the architecture of PixelCNN should look like, however, it does a good job of describing the big picture, but it is not enough for actually implementing PixelCNN.

Here’s the architecture I came up with for grayscale MNIST (with only 1 residual block for simplicity):

<img src=https://israelg99.github.io/images/2017-02-27-Grayscale-PixelCNN-with-Keras/model.png>

Note that PixelCNN has to preserve the spatial dimension of the input, which is not shown in the graph above.

<a id="MaskedConvolution"></a>
## Masked Convolution

We already defined our input, and as you can see in the architecture graph, the next layer is a masked convolution, which is the next thing we are going to implement.

### How to implement grayscale masks?

Here’s a picture for reference:

<img src=https://israelg99.github.io/images/2017-02-27-Grayscale-PixelCNN-with-Keras/grayscale_mask_typeA.png>

The difference between type A and B masks in grayscale images is that type A also masks the center pixel.
Keep in mind that masks for grayscale images are simpler than RGB masks, but we’ll get to RGB masks too.

Here’s how we are going to implement masks:

1. Create a numpy array of ones in the shape of our convolution weights: (height, width, input_channels, output_channels)
2. Zero out all weights to the right and below of the center weights (to block future insight of pixels from flowing, as stated in the paper).
3. If the mask type is A, we’ll zero out the center weights too (to block insight of the current pixel as well).
4. Multiply the mask with the weights before calculating convolutions.

Let’s use the steps above to go ahead and implement a new Keras layer for masked convolutions:

In [24]:
import math

import numpy as np

from keras import backend as K
from keras.layers import Convolution2D
from keras import Input

In [30]:
class MaskedConvolution2D(Convolution2D):
    #*args pick up any number of non-keyword arguments
    #*kwargs pick up any number of keyword arguments that are actually dictionaries
    def __init__(self, *args, mask='B' , n_channels=3, mono=False, **kwargs):
        super().__init__(*args, **kwargs)
        self.mask_type = mask

        self.mask = None
        
    def build(self, input_shape):
        super().build(input_shape)

        # Create a numpy array of ones in the shape of our convolution weights.
        self.mask = np.ones(self.kernel.shape)
        #print(self.mask, "is of type", type(self.mask))
        #self.mask = np.ones(np.array(self.get_weights()).shape)
        #self.mask = np.ones((np.array(self.get_weights()).shape), (np.array(self.get_weights()).shape))

        # We assert the height and width of our convolution to be equal as they should.
        assert self.mask.shape[0] == self.mask.shape[1]

        # Since the height and width are equal, we can use either to represent the size of our convolution.
        filter_size = self.mask.shape[0]
        filter_center = filter_size / 2

        # Zero out all weights below the center.
        self.mask[math.ceil(filter_center):] = 0

        # Zero out all weights to the right of the center.
        self.mask[math.floor(filter_center):, math.ceil(filter_center):] = 0

        # If the mask type is 'A', zero out the center weigths too.
        if self.mask_type == 'A':
            self.mask[math.floor(filter_center), math.floor(filter_center)] = 0

        # Convert the numpy mask into a tensor mask.
        self.mask = K.variable(self.mask)
    
    def call(self, x, mask=None):
        ''' I just copied the Keras Convolution2D call function so don't worry about all this code.
            The only important piece is: self.W * self.mask.
            Which multiplies the mask with the weights before calculating convolutions. '''
        output = K.conv2d(x, self.kernel * self.mask, strides=self.strides,
                          padding=self.padding,
                          data_format=self.data_format,
                          filter_shape=self.kernel.shape)
                          
        if self.use_bias:
            #Dimension ordering th means the channel dimension (the depth) is at index 1.
            #nb_filter is the number of convolutional filters to use.
            if self.data_format == 'th':
                output += K.reshape(self.b, (1, self.nb_filter, 1, 1))
            #Dimension ordering th means the channel dimension (the depth) is at index 3.
            elif self.data_format == 'tf':
                output += K.reshape(self.b, (1, 1, 1, self.nb_filter))
            else:
            #There are no other kind of dimension orderings so any other case would be invalid.
                raise ValueError('Invalid data_format:', self.data_format)

        output = self.activation(output)
        return output

    def get_config(self):
        # Add the mask type property to the config.
        return dict(list(super().get_config().items()) + list({'mask': self.mask_type}.items()))

<a id="FirstMaskedConvolution"></a>
## First Masked Convolution Layer

Now that we have masked convolutions implemented, let’s add the first masked convolution to our model(which is practically just an input layer at the moment).

According to the paper, the layer after the input is a masked convolution of type A, with a filter size of (7,7) and it has to preserve the spatial dimensions of the input, we’ll use border_mode='same' for that.
Note that this layer is the only masked convolution of type A the model will have.

Now we should have a simple graph like this: input -> masked_convolution.

<a id="KerasInput"></a>
### Keras.Input

Input() is used to instantiate a Keras tensor.

A Keras tensor is a TensorFlow symbolic tensor object, which we augment with certain attributes that allow us to build a Keras model just by knowing the inputs and outputs of the model.

For instance, if a, b and c are Keras tensors, it becomes possible to do: model = Model(input=[a, b], output=c)

#### Arguments

* **shape:** A shape tuple (integers), not including the batch size. For instance, shape=(32,) indicates that the expected input will be batches of 32-dimensional vectors. Elements of this tuple can be None; 'None' elements represent dimensions where the shape is not known.
* **batch_size:** optional static batch size (integer).
* **name:** An optional name string for the layer. Should be unique in a model (do not reuse the same name twice). It will be autogenerated if it isn't provided.
* **dtype:** The data type expected by the input, as a string (float32, float64, int32...)
* **sparse:** A boolean specifying whether the placeholder to be created is sparse. Only one of 'ragged' and 'sparse' can be True. Note that, if sparse is False, sparse tensors can still be passed into the input - they will be densified with a default value of 0.
* **tensor:** Optional existing tensor to wrap into the Input layer. If set, the layer will not create a placeholder tensor.
* **ragged:** A boolean specifying whether the placeholder to be created is ragged. Only one of 'ragged' and 'sparse' can be True. In this case, values of 'None' in the 'shape' argument represent ragged dimensions. For more information about RaggedTensors, see this guide.
* **kwargs:** deprecated arguments support. Supports batch_shape and batch_input_shape.

In [31]:
shape = (28, 28, 1)
filters = 128

#Depth required when stacking residual blocks
depth = 6

input_img = Input(shape)

model = MaskedConvolution2D(filters, 7, 7, mask='A', padding='same')(input_img)

ValueError: It seems that you are using the Keras 2 and you are passing both `kernel_size` and `strides` as integer positional arguments. For safety reasons, this is disallowed. Pass `strides` as a keyword argument instead.

<a id="ResidualBlocks"></a>
## Residual blocks

After the first masked convolution the model has a series of residual blocks (The architecture picture above has only 1 residual block).

To implement a residual block:

1. Take input of shape (height, width, filters).
2. Halve the filters with a (1,1) convolution.
3. Apply a (3,3) masked convolution of type B.
4. Scale the filters back to original with (1,1) convolution.
5. Merge the original input with the convolutions.

The reason for cutting the filters by half and then scaling back to original is because it is a good way to get a computational boost while not significally reducing model performance.

Let’s implement a residual block in Keras:

In [ ]:
class ResidualBlock(object):
    def __init__(self, filters):
        self.filters = filters

    def __call__(self, model):
        # filters -> filters/2
        block = PReLU()(model)
        block = Convolution2D(self.filters//2, 1, 1)(block)

        # filters/2 3x3 -> filters/2
        block = PReLU()(block)
        block = MaskedConvolution2D(self.filters//2, 3, 3, border_mode='same')(block)

        # filters/2 -> filters
        block = PReLU()(block)
        block = Convolution2D(self.filters, 1, 1)(block)

        # Merge the original input with the convolutions.
        return Merge(mode='sum')([model, block])

<a id="PRelu"></a>
### PRelu Layers

The parametric rectifier linear unit (pReLU) activation layer applies the transform f(x) = max(0, x) + w * min(0, x) to the input data. The backward pReLU layer computes the values z = y*f'(x), where y is the input gradient computed on the preceding layer, w is the weight of the input argument. and

<img src=https://software.intel.com/sites/products/documentation/doclib/daal/daal-user-and-reference-guides/daal_prog_guide/equations/GUID-ADC54AE0-43B8-40CA-BA41-245D3240Bee1.png>

#### Parameters
* **alpha_initializer:** Initializer function for the weights.
* **alpha_regularizer:** Regularizer for the weights.
* **alpha_constraint:** Constraint for the weights.
* **shared_axes:** The axes along which to share learnable parameters for the activation function. For example, if the incoming feature maps are from a 2D convolution with output shape (batch, height, width, channels), and you wish to share parameters across space so that each filter only has one set of parameters, set shared_axes=[1, 2].

We will want to stack those residual blocks in our model, so let’s create a simple layer for that:

In [ ]:
class ResidualBlockList(object):
    def __init__(self, filters, depth):
        self.filters = filters
        self.depth = depth

    def __call__(self, model):
        for _ in range(self.depth):
            model = ResidualBlock(self.filters)(model)

        return model

<a id="StackingResidualBlocks"></a>
## Stacking Residual Blocks

Now let’s stack those residual blocks on our model.
We also need to add an activation after the stack, because the residual block ends with a convolution, not an activation.

In [ ]:
#shape = (28, 28, 1)
#filters = 128
#depth = 6

#input_img = Input(shape)

#model = MaskedConvolution2D(filters, 7, 7, mask='A', border_mode='same')(input_img)

model = ResidualBlockList(filters, depth)
model = PReLU()(model)

<a id="WrappingUpForOutput"></a>
## Wrapping Up For Output

As shown in the architecture picture above, the model has additional 2 masked convolutions before output. According to the paper, those 2 masked convolutions are of size (1,1) and of type B.

Let’s add those to our model:

In [ ]:
#shape = (28, 28, 1)
#filters = 128
#depth = 6

#input_img = Input(shape)

#model = MaskedConvolution2D(filters, 7, 7, mask='A', border_mode='same')(input_img)

#model = ResidualBlockList(filters, depth)
#model = PReLU()(model)

for _ in range(2):
    model = MaskedConvolution2D(filters, 1, 1, border_mode='valid')(model)
    model = PReLU()(model)

<a id="Output"></a>
## Output

Since we have just one channel, we can convolve the pixels with a convolution of size (1,1) with a single filter and then sigmoid its output.
The output of the sigmoid should be a 2D array with an exact shape as the input ((28, 28, 1) for MNIST), with each point in the 2D array representing a (grayscale) color value.

It shoud look like this:

In [ ]:
#shape = (28, 28, 1)
#filters = 128
#depth = 6

#input_img = Input(shape)

#model = MaskedConvolution2D(filters, 7, 7, mask='A', border_mode='same')(input_img)

#model = ResidualBlockList(filters, depth)
#model = PReLU()(model)

#for _ in range(2):
#    model = MaskedConvolution2D(filters, 1, 1, border_mode='valid')(model)
#    model = PReLU()(model)

outs = Convolution2D(1, 1, 1, border_mode='valid')(model)
outs = Activation('sigmoid')(outs)

<a id="CompilingTheModel"></a>
## Compiling the Model

I chose Nadam quite arbitrarily, you can go with any optimizer you like.
Since we use sigmoid in our output activations our loss should be binary_crossentropy.

Let’s compile the model:

In [ ]:
model = Model(input_img, outs)
model.compile(optimizer=Nadam(), loss='binary_crossentropy')

<a id="TrainingTheModel"></a>
## Training the Model
<a id="LoadingMNIST"></a>
#### Loading MNIST data with Keras

Let’s load MNIST data, change data type to float and normalize it.
We ignore the labels of MNIST because they are not useful for our case, our model is not a classifier.

Note that I concatenate the training and test data, I do that to have more data to help with training, however, if you need validation data, feel free to not concatenate them.

In [ ]:
(train, _), (test, _) = mnist.load_data()
data = np.concatenate((train, test), axis=0)
data = data.astype('float32')
data /= 255

<a id="FitMNIST"></a>
#### Fit the Model to MNIST

Set the arguments needed for training, I also added a TensorBoard and a ModelCheckpoint callbacks to the training routine.
We pass the data both as our input and target output.

This setup is similar to an autoencoder’s one, but PixelCNN is not an autoencoder, it doesn’t learn an effcient encoding of the data but rather learns the distribution of the values in the data.

Here’s how the fit routine should look like:

In [ ]:
batch_size = 32
epochs = 200
callbacks = [TensorBoard(), ModelCheckpoint('model.h5')]

model.fit(data, data,
      batch_size=batch_size, nb_epoch=epochs,
      callbacks=callbacks)

<a id="GenerateMNIST"></a>
#### Generating MNIST

To generate images PixelCNN needs to predict each pixel separately, more concretly, you will need to feed it an 2D array of zeros, let it predict the first pixel, refeed the array, predict second pixel, and so on…

PixelCNN outputs an array of pixel value probabilities for each pixel, to generate different images we should not use argmax when picking a pixel value, but rather pick a pixel value using the probabilities themselves.

Keep in mind, generating images does not benefit from the convolution’s concurrent nature and everything has to be done sequentially pixel by pixel.

In [ ]:
# Using a batch size allows us to predict a few pixels concurrently.
batch = 8

# Create an empty array of pixels.
pixels = np.zeros(shape=(batch,) + (model.input_shape)[1:])
batch, rows, cols, channels = pixels.shape

# Iterate the pixels because generation has to be done sequentially pixel by pixel.
for row in range(rows):
    for col in range(cols):
        for channel in range(channels):
            # Feed the whole array and retrieving the pixel value probabilities for the next pixel.
            ps = model.predict_on_batch(pixels)[channel][:, row*cols+col]

            # Use the probabilities to pick a pixel value.
            # Lastly, we normalize the value.
            pixels[:, row, col, channel] = np.array([np.random.choice(256, p=p) for p in ps]) / 255

# Iterate the generated images and plot them with matplotlib.
for pic in pixels:
    plt.imshow(pic, interpolation='nearest')
    plt.show(block=True)